In [54]:
import torch
import pandas as pd
import numpy as np
import cv2

class Zoom_and_rotate:
    def __init__(self):
        self.model_yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
       


    def detect_main_car(self , img):
        results = self.model_yolov5(img)
        cars0 = results.pandas().xyxy[0][results.pandas().xyxy[0]['name'] == 'car']
        vans = results.pandas().xyxy[0][results.pandas().xyxy[0]['name'] == 'van']
        trucks = results.pandas().xyxy[0][results.pandas().xyxy[0]['name'] == 'truck']
        cars = pd.concat([cars0, vans , trucks], axis=0)
        cars.reset_index(drop=True, inplace=True)
        if len(cars) == 0:
            return None
        biggest_car = cars.iloc[(cars['xmax'] - cars['xmin']).argmax()]

        if biggest_car['name'] == 'truck' or  biggest_car['name'] == 'van':
            center_bb_x1 = img.shape[1] * 0.25
            center_bb_x2 = img.shape[1] * 0.75
            center_bb_y1 = img.shape[0] * 0.25
            center_bb_y2 = img.shape[0] * 0.75

            car_center_x = (biggest_car['xmax'] - biggest_car['xmin']) / 2
            car_center_y = (biggest_car['ymax'] - biggest_car['ymin']) / 2

            if (center_bb_x1 <= car_center_x <= center_bb_x2)  and (center_bb_y1 <= car_center_y <= center_bb_y2) :
                biggest_car = biggest_car
            else:
                cars = cars0
                if len(cars) == 0:
                    return None
                biggest_car = cars.iloc[(cars['xmax'] - cars['xmin']).argmax()]
        return biggest_car
    
    def make_new_image(self , img , bb , expected_car_heigh):
        new_width = img.shape[1] + 6000
        new_height = img.shape[0] + 6000
        new_img = np.zeros((new_height, new_width, 3), np.uint8)
        new_img[3000:new_height-3000, 3000:new_width-3000] = img

        current_height = bb['ymax'] - bb['ymin']
        scale = current_height / expected_car_heigh
        cut_size_y , cut_size_x =  scale * img.shape[0] , scale * img.shape[1]
        car_center_x = int((bb['xmax'] + bb['xmin']) / 2) + 3000
        car_center_y = int((bb['ymax'] + bb['ymin']) / 2) + 3000

        cut_img = new_img[int(car_center_y - cut_size_y / 2):int(car_center_y + cut_size_y / 2) , int(car_center_x - cut_size_x / 2):int(car_center_x + cut_size_x / 2),:]
        return   cv2.resize(cut_img, ( img.shape[1] , img.shape[0]), interpolation=cv2.INTER_LINEAR)




    def  zoomIN_zoomOut(self , img , height_fraction = 0.4 ):
        bb = self.detect_main_car(img)
        h = bb['ymax'] - bb['ymin']
        if (height_fraction - 0.01) * img.shape[0] < h < (height_fraction + 0.01) * img.shape[0]:
            return img
        return self.make_new_image(img, bb , int(height_fraction * img.shape[0]) )
    
    def rotate_image(self , img, angle):
        height, width = img.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
        rotated_image = cv2.warpAffine(img, rotation_matrix, (width, height))
        return rotated_image
        




In [55]:
import cv2

im = cv2.imread("top.png")

In [56]:
obj = Zoom_and_rotate()

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


Adding AutoShape... 


In [57]:
z = 0.05
for i in range(20):
    img2 = obj.zoomIN_zoomOut(im , z)
    z += 0.02
    cv2.imwrite("zoom"+str(i)+".png" , img2)

In [64]:
import glob
i = 0 
for im in glob.glob("IMG_4384__8483/*"):
    i+=1
    img = cv2.imread(im)
    img2 = obj.zoomIN_zoomOut(img , 0.45)
    cv2.imwrite("Zoom/zoom_"+im.split("\\")[-1].split(".")[0].split("_")[1]+".png" , img2)

In [77]:
aa = obj.rotate_image(cv2.imread("IMG_4384__8483/image_50.png") ,8 )

In [78]:
cv2.imwrite("rotate/8.png" , aa)

True